### process_data_raw_text_to_DB

 Extract date, title, author and text from each web page scraped by scrape_american_presidency_raw_text.ipynb and insert the record into mongoDB

In [1]:
import pymongo
from pymongo import MongoClient
import datetime
import pickle

import re
from dateutil import parser


#### connect to database; start a collection to store records with author, title, date and raw text as fields.

In [2]:
client = MongoClient()

In [3]:
db = client.test_database

In [ ]:
presidency_docs = db.presidency_docs

In [7]:
'''
Extract date, title, author and text from a scraped web page
return a mongo DB record.

'''

def process_docs (docs):

    mongo_db_records = []

    # a very small fraction of pages have a different format:
    # author, title, date are not included in <span class="displaytext">
    # but rather, each is in a seperate <span class="ver10">
    # that follows <span class="displaytext">
    # mark those pages and process them differently later
    error_ids = []

    for i in range(len(docs)):
        content = docs[i]
        text = content.split('Online by')[0]
    
        # get rid of '[laughter]', '[applause]','(applause)', etc.
        text = re.sub('\[.*?\]', ' ', text)
        text = re.sub('\(.*?\)', ' ', text)
    
        # get rid of footnotes
        lst =text.split('\n\tCitation:\xa0')
    
    
        doc=lst[0]
        doc=doc.split('Note:')[0]
        doc=doc.split('NOTE:')[0]
        doc=doc.split('NOTE:')[0]
    
    
        try:
            if len(lst)>1:
    
                temp=lst[1].split('"')
                title = temp[1].strip().replace(',','')
                date = temp[2].strip().replace('.','')
                name=temp[0].strip().replace(':','')
    
    
                record = {"author":name,"title":title,"date":parser.parse(date),"text":doc}
                mongo_db_records.append(record)
        
            else:
                error_ids.append(i)
        except:
            error_ids.append(i)
    
    
    
    return mongo_db_records ,error_ids

#### now precess all 130K raw text and save page ids of the ones that fail

In [22]:
def process_all_raw_text_filenames():
        
    start_pts = [i for i in range(1,128000,1000)]
    end_pts = [ i+1000 for i in start_pts]
    last_interval = (129001, 129484)

    start_pts.append(last_interval[0])
    end_pts.append(last_interval[1])
    
    all_err_ids = []
    
    for i in range(len(start_pts)):
        filename = '/home/ubuntu/proj4/data/presidency_docs_'+str(start_pts[i])+'_to_'+str(end_pts[i]-1)+'.pkl'
        with open(filename, 'rb') as fp:
            docs = pickle.load(fp)
        # err_ids are the positions of error pages in current doc list    
        records, err_ids = process_docs (docs)
        # err_page_ids are the page ids the web site uses for these pages
        err_page_ids = [err_id+start_pts[i] for err_id in err_ids]
        all_err_ids.extend(err_page_ids)
        presidency_docs.insert_many(records)
        
        print('inserted docs ',start_pts[i],' to ',end_pts[i]-1, )
            
    return all_err_ids

In [23]:
err_page_ids = process_all_raw_text_filenames()

inserted docs  1  to  1000
inserted docs  1001  to  2000
inserted docs  2001  to  3000
inserted docs  3001  to  4000
inserted docs  4001  to  5000
inserted docs  5001  to  6000
inserted docs  6001  to  7000
inserted docs  7001  to  8000
inserted docs  8001  to  9000
inserted docs  9001  to  10000
inserted docs  10001  to  11000
inserted docs  11001  to  12000
inserted docs  12001  to  13000
inserted docs  13001  to  14000
inserted docs  14001  to  15000
inserted docs  15001  to  16000
inserted docs  16001  to  17000
inserted docs  17001  to  18000
inserted docs  18001  to  19000
inserted docs  19001  to  20000
inserted docs  20001  to  21000
inserted docs  21001  to  22000
inserted docs  22001  to  23000
inserted docs  23001  to  24000
inserted docs  24001  to  25000
inserted docs  25001  to  26000
inserted docs  26001  to  27000
inserted docs  27001  to  28000
inserted docs  28001  to  29000
inserted docs  29001  to  30000
inserted docs  30001  to  31000
inserted docs  31001  to  3200

inserted docs  90001  to  91000
inserted docs  91001  to  92000
inserted docs  92001  to  93000
inserted docs  93001  to  94000
inserted docs  94001  to  95000
inserted docs  95001  to  96000
inserted docs  96001  to  97000
inserted docs  97001  to  98000
inserted docs  98001  to  99000
inserted docs  99001  to  100000
inserted docs  100001  to  101000
inserted docs  101001  to  102000
inserted docs  102001  to  103000
inserted docs  103001  to  104000
inserted docs  104001  to  105000
inserted docs  105001  to  106000
inserted docs  106001  to  107000
inserted docs  107001  to  108000
inserted docs  108001  to  109000
inserted docs  109001  to  110000
inserted docs  110001  to  111000
inserted docs  111001  to  112000
inserted docs  112001  to  113000
inserted docs  113001  to  114000
inserted docs  114001  to  115000
inserted docs  115001  to  116000
inserted docs  116001  to  117000
inserted docs  117001  to  118000
inserted docs  118001  to  119000
inserted docs  119001  to  120000